In [68]:
import pandas as pd
import numpy as np

In [69]:
cyber = pd.read_csv('CSIC.csv')


In [70]:
cyber.head()

,method,url,payload,payload_len,alpha,non_alpha,attack_feature,label
0,0,http://localhost:8080/tienda1/publico/caracter...,modo=entrar&login=hazell&pwd=clam%21o&remember...,59,96.153846,3.846154,0.040000,0
1,0,http://localhost:8080/tienda1/publico/caracter...,modo=entrar&login=sagers&pwd=C%21HuZAr&remembe...,60,96.296296,3.703704,0.038462,0
2,0,http://localhost:8080/tienda1/publico/caracter...,modo=entrar&login=verde&pwd=en683%2467&remembe...,60,96.296296,3.703704,0.038462,0
3,0,http://localhost:8080/tienda1/publico/caracter...,modo=entrar&login=nolie&pwd=nort%24e%D1a&remem...,62,96.296296,3.703704,0.038462,0
4,0,http://localhost:8080/tienda1/publico/caracter...,modo=entrar&login=niu&pwd=raqu%24%ED9omo&remem...,62,96.296296,3.703704,0.038462,0


In [71]:
cyber['payload'] = cyber['payload'].fillna(' ')


In [72]:
cyber.isnull().sum()

method            0
url               0
payload           0
payload_len       0
alpha             0
non_alpha         0
attack_feature    0
label             0
dtype: int64

In [73]:
# Remove column name 
cyber.drop(['url'], axis=1,inplace=True)

In [74]:
cyber.head()

,method,payload,payload_len,alpha,non_alpha,attack_feature,label
0,0,modo=entrar&login=hazell&pwd=clam%21o&remember...,59,96.153846,3.846154,0.040000,0
1,0,modo=entrar&login=sagers&pwd=C%21HuZAr&remembe...,60,96.296296,3.703704,0.038462,0
2,0,modo=entrar&login=verde&pwd=en683%2467&remembe...,60,96.296296,3.703704,0.038462,0
3,0,modo=entrar&login=nolie&pwd=nort%24e%D1a&remem...,62,96.296296,3.703704,0.038462,0
4,0,modo=entrar&login=niu&pwd=raqu%24%ED9omo&remem...,62,96.296296,3.703704,0.038462,0


In [75]:
from sklearn.model_selection import train_test_split

attributes = ['method', 'payload', 'payload_len', 'alpha', 'non_alpha','attack_feature']
x_train, x_test, y_train, y_test = train_test_split(cyber[attributes], cyber['label'], test_size=0.2, 
                                                    stratify=cyber['label'], random_state=0)

In [76]:
cyber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35602 entries, 0 to 35601
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          35602 non-null  int64  
 1   payload         35602 non-null  object 
 2   payload_len     35602 non-null  int64  
 3   alpha           35602 non-null  float64
 4   non_alpha       35602 non-null  float64
 5   attack_feature  35602 non-null  float64
 6   label           35602 non-null  int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 1.9+ MB


In [77]:
x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.3, 
                                                  stratify=y_train, random_state=0)
print('Train:', len(y_train), 'Dev:', len(y_dev), 'Test:', len(y_test))

Train: 19936 Dev: 8545 Test: 7121


In [78]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(analyzer='word', min_df=10)
n_grams_train = count_vectorizer.fit_transform(x_train['payload'])
n_grams_dev = count_vectorizer.transform(x_dev['payload'])

print('Number of features:', len(count_vectorizer.vocabulary_))

Number of features: 565


### SGDClassifier

In [86]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

sgd = SGDClassifier(random_state=0)
sgd.fit(n_grams_train, y_train)
y_pred_sgd = sgd.predict(n_grams_dev)
print("SGDClassifier accuracy:", accuracy_score(y_dev, y_pred_sgd))

SGDClassifier accuracy: 0.9885313048566413


### Logistic Regression

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lgr = LogisticRegression()
lgr.fit(n_grams_train, y_train)
y_pred_lgr = lgr.predict(n_grams_dev)
print("Logistic Regression accuracy:", accuracy_score(y_dev, y_pred_sgd))

Logistic Regression accuracy: 0.9885313048566413


C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### K Nearest Neighbor

In [88]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 15)
kn.fit(n_grams_train, y_train)
y_pred_kn = kn.predict(n_grams_dev)
print("KNeighborsC;assifier accuracy:", accuracy_score(y_dev, y_pred_sgd))


KNeighborsC;assifier accuracy: 0.9885313048566413


### Support Vector Machine

In [89]:
from sklearn.svm import SVC
svm = SVC(kernel='linear',C=0.025,random_state = 100)
svm.fit(n_grams_train, y_train)
y_pred_svm = svm.predict(n_grams_dev)

print("Support Vector Machine accuracy:", accuracy_score(y_dev, y_pred_sgd))

Support Vector Machine accuracy: 0.9885313048566413


### K Nearest Neighbor

In [90]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 15)
kn.fit(n_grams_train, y_train)
y_pred_kn = kn.predict(n_grams_dev)
print("KNeighborsC;assifier accuracy:", accuracy_score(y_dev, y_pred_sgd))


KNeighborsC;assifier accuracy: 0.9885313048566413


### Support Vector Machine

In [91]:
from sklearn.svm import SVC
svm = SVC(kernel='linear',C=0.025,random_state = 100)
svm.fit(n_grams_train, y_train)
y_pred_svm = svm.predict(n_grams_dev)

print("Support Vector Machine accuracy:", accuracy_score(y_dev, y_pred_sgd))


Support Vector Machine accuracy: 0.9885313048566413
